Read US Cities CSV file and put each city/state into a list

In [2]:
import csv 

filename = 'uscities.csv'
cities = []

with open(filename,'r') as file:
    reader = csv.reader(file)
    next(reader)
    for line in reader:
        city = line[0].replace(' ','-')
        city = city.replace('St.','Saint')
        state = line[2]
        cities.append((city,state))
        

In [ ]:
import requests
import sys
from bs4 import BeautifulSoup
import csv
from concurrent.futures import ThreadPoolExecutor, as_completed
from queue import Queue
import threading

cost_living_csv = 'cost_of_living.csv'
cities_added = 'cities_add.csv'
header_made = 0
write_queue = Queue()
header_written = threading.Event()
cities = cities[1350:]

#Filter through the html to get the table data, then add it to the csv
def add_city_data(response,city_name,state):
    global header_made
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    table = soup.find('table', class_='data_wide_table')
    if not table: 
        return #end if table does not exist

    row_data = [city_name,state]
    header = ['City','State']

    for row in table.find_all('tr'):
        cols = row.find_all('td')
        data = [td.get_text(strip=True) for td in cols]
        if len(data) > 2:
            item = data[0].strip()
            avg_price = data[1].replace('\xa0$','').strip()
            price_range = data[2].strip()
            prices = (avg_price,price_range)
            row_data.append(prices)
            header.append(item)

    with open(cost_living_csv, 'a', newline = '') as file:
        writer = csv.writer(file)
        if not header_made:
            writer.writerow(header)
            header_made = 1
        writer.writerow(row_data)
        file.flush()

#Create URL from the city/state and retreive website response, call add_city_data function with response as an input
def fetch_data(city,state):
    city_name = city.replace('-',' ')
    url = f'https://www.numbeo.com/cost-of-living/in/{city}'
    backup_url = f'https://www.numbeo.com/cost-of-living/in/{city}-{state}-United-States'
    urls = [url,backup_url]

    for url in urls:
        try:
            response = requests.get(url)
            if response.status_code == 200:
                add_city_data(response,city_name,state)
        except:
                sys.stderr.write(f'Cannot retreive data for {city_name}\n')

for city, state in cities:
    fetch_data(city,state)

Cannot retreive data for Spanaway
Cannot retreive data for Lakeside
Cannot retreive data for Cheney
Cannot retreive data for Santa Fe
Cannot retreive data for Santa Fe
Cannot retreive data for Stony Brook
Cannot retreive data for Stony Brook
Cannot retreive data for Forrest City
Cannot retreive data for Forrest City
Cannot retreive data for Amherst
Cannot retreive data for Amherst
Cannot retreive data for Florida City
Cannot retreive data for Florida City
Cannot retreive data for Vadnais Heights
Cannot retreive data for Vadnais Heights
Cannot retreive data for Woodhaven
Cannot retreive data for Woodhaven
Cannot retreive data for Branson
Cannot retreive data for Branson
Cannot retreive data for Emeryville
Cannot retreive data for Emeryville
Cannot retreive data for Greenville
Cannot retreive data for Greenville
Cannot retreive data for Gonzales
Cannot retreive data for Gonzales
Cannot retreive data for Sandy
Cannot retreive data for Sandy
Cannot retreive data for Manorville
Cannot retre

In [ ]:
quality_life_csv = 'cost_of_living.csv'
header_made = 0
# cities = cities[101:]

#Filter through the html to get the table data, then add it to the csv
def add_quality_life_data(response,city_name,state):
    global header_made
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    table = soup.find('table')
    if not table: 
        return #end if table does not exist

    row_data = [city_name,state]
    header = ['City','State']

    for row in table.find_all('tr'):
        cols = row.find_all('td')
        data = [td.get_text(strip=True) for td in cols]
        # if len(data) > 2:
        #     item = data[0].strip()
        #     avg_price = data[1].replace('\xa0$','').strip()
        #     price_range = data[2].strip()
        #     prices = (avg_price,price_range)
        #     row_data.append(prices)
        #     header.append(item)

    with open(cost_living_csv, 'a', newline = '') as file:
        writer = csv.writer(file)
        if not header_made:
            writer.writerow(header)
            header_made = 1
        writer.writerow(row_data)
        file.flush()

#Create URL from the city/state and retreive website response, call add_quality_life_data function with response as an input
def fetch_quality_life_data(city,state):
    city_name = city.replace('-',' ')
    url = f'https://www.numbeo.com/quality-of-life/in/{city}'
    backup_url = f'https://www.numbeo.com/quality-of-life/in/{city}-{state}-United-States'
    urls = [url,backup_url]

    for url in urls:
        try:
            response = requests.get(url)
            if response.status_code == 200:
                add_city_data(response,city_name,state)
        except:
                sys.stderr.write(f'Cannot retreive data for {city_name}\n')

for city, state in cities:
    fetch_quality_life_data(city,state)

In [ ]:
import json
import requests

api_key = '688f5866ca7e4ed890d5798ba3f8ef29'
endpoint = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

def get_BLS_data(seriesId, **kwargs):
    #Pass BLS timeseries data to return data in JSON format, arguments can be added to specify search
    if len(seriesId) < 1 or len(seriesId) > 25:
        raise ValueError('SeriesId must be at least 1 and less than 25')
    
    headers = {'Content-Type': 'application/json'}
    payload = {
        'seriesid': seriesId,
        'registrationKey': api_key
    }
    #Update payload with keyword arguments and convert to JSON
    payload.update(kwargs)
    payload = json.dumps(payload)

    #Get request to BLS API
    response = requests.post(endpoint, data = payload, headers = headers)
    response.raise_for_status()

    result = json.loads(response.text)
    if result['status'] != 'REQUEST_SUCCEEDED':
        raise Exception(result['message'][0])
    return result

seriesId = ['CEU5051612001']
json_data = get_BLS_data(seriesId, startyear = 2015, endyear = 2024)
print(json_data)

    


